In [1]:

import os
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
from matplotlib import gridspec
import numpy as np
import sys
from itertools import compress
import matplotlib.colors as clr
from math import pi


sys.path.append("scripts/")
import scripts.conf as conf
import scripts.oncotree
conf.config_params()
oncotree = scripts.oncotree.Oncotree()

os.makedirs("raw_plots",exist_ok=True)
os.makedirs("source_data",exist_ok=True)

# Extended Figure 5, radar plots.

### Function to plot radar for a single mutation

In [20]:
def plot_radar_single_mutation(features,values,drivers,path_output):
    # Definition of variables 
    fig = plt.subplots(figsize=(1.,1.))
    categories=features
    degron = "_GLOBAL_VALUE"

    N = len(categories)

    # What will be the angle of each axis in the plot? (we divide the plot / number of variable)
    angles = [n / float(N) * 2 * pi for n in range(N)]
    angles += angles[:1]
    # Initialise the spider plot
    ax = plt.subplot(111, polar=True)
    ax.spines['polar'].set_visible(False)
    # If you want the first axis to be on top:
    ax.set_theta_offset(pi / 2)
    ax.set_theta_direction(-1)
    ax.vlines(angles[:-1], 0, [3 for x in range(len(angles[:-1]))], colors="#DCDCDC", zorder=3,linewidth=0.5,alpha=0.5) 
    # Draw ylabels
    ax.set_rlabel_position(0)
    maxv = int(np.amax(values))+1
    minv = int(np.amin(values))-1
    plt.ylim(minv,maxv)
    plt.yticks(np.arange(minv,maxv+1,1), np.arange(minv,maxv+1,1), color="black", size=5)
    values +=  values[:1]
    ax.plot(angles, values, linewidth=1.15, linestyle='solid', label="Drivers",color="#a64040ff")
    ax.grid(b=None,axis="x")
    ax.grid(axis="y",lw=0.25)
    
    # Draw one axe per variable + add labels labels yet
    _=plt.xticks(angles[:-1], [conf.name_features[x]  for x in features],fontsize=4)
    plt.savefig(path_output,dpi=800,bbox_inches="tight")
    


In [21]:
path_saturation=os.path.join(conf.output_boostdm,"saturation","prediction")

### Individual examples, Extended Figure 5 a)

In [22]:
examples = [("EGFR","A289D","GBM"),
           ("VHL","S80R","RCCC"),("CTNNB1","S45P","HC")]

In [23]:
for gene,mut,ttype in examples:
    path_file = os.path.join(path_saturation,f"{gene}.{ttype}.prediction.tsv.gz")
    df_data = pd.read_csv(path_file,sep="\t")
    df_data[(df_data["boostDM_class"])&(df_data["gene"]==gene)&(df_data["aachange"]==mut)].to_csv(f"source_data/{gene}_{ttype}_{mut}_radar_source.tsv",sep="\t")
    muts = df_data[(df_data["boostDM_class"])&(df_data["gene"]==gene)&(df_data["aachange"]==mut)]["aachange"].unique()
    for mut in muts:
        df = df_data[df_data["aachange"]==mut].drop_duplicates()
        for i,r in df.iterrows():
            selected_model = r["selected_model_ttype"]
            selected_model_gene = r["selected_model_gene"]
            gene = r["gene"]
            values = [r.loc[feature] for feature in conf.features]
            drivers = [r["boostDM_class"]]
            name = os.path.join("raw_plots",gene+"_"+mut+"_"+ttype+".svg")
            plot_radar_single_mutation(conf.features,values,drivers,name)

### Individual examples, Extended Figure 5 a); splice site mutation

In [24]:
gene="TP53"
mutations = ["."]
ttype = "ESCA"
path_file = os.path.join(path_saturation,f"{gene}.{ttype}.prediction.tsv.gz")
df_data = pd.read_csv(path_file,sep="\t")
df = df_data[(df_data["boostDM_class"])&(df_data["gene"]==gene)&(df_data["pos"].isin([7674859]))&(df_data["alt"]=="T")]
mut="7674859-C>T"
df.to_csv(f"source_data/{gene}_{ttype}_{mut}_radar_source.tsv",sep="\t")
for i,r in df.iterrows():
    values = [r.loc[feature] for feature in conf.features]
    drivers = [r["boostDM_class"]]
    
    name = os.path.join("raw_plots",gene+"_"+mut+"_"+ttype+".svg")
    plot_radar_single_mutation(conf.features,values,drivers,name)